In [ ]:

В базе данных вашей компании присутствуют следующие таблицы

city

city_id – id города, где проходит акция
client_city_id – id города, где находится покупатель
city – город
client_city – город, где находится покупатель


client

client_id – id покупателя
client_city_id – id города, где находится покупатель
birth_date – дата рождения покупателя
registration – дата регистрации покупателя


promotion

promotion_id – id акции
category_id – id категории
promotion_name – акция
category_name – категория
partner_id – id партнёра
partner_name – партнёр


purchase

purchase_id – id покупки
partner_id – id партнёра
client_id – id покупателя
city_id – id города
promotion_id – id акции
category_id – id категории
purchase_date – дата покупки
price – цена за единицу товара
quantity – число проданных единиц
status – статус покупки

Вам нужно написать запрос, чтобы получить такую таблицу

purchase_date – дата покупки
purchase_id – id покупки
client_id – id покупателя
client_age – возраст покупателя
client_registration_age – как долго человек пользуется вашими магазинами
client_category – содержит new или old, если это первая или последующая покупка соответственно
promotion_name – акция
category_name – категория
partner_name – партнёр
client_city – город, где находится покупатель
city – город
revenue – сумма выручки
quantity – число проданных единиц

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, 
и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

In [1]:
import pandas as pd
import pandahouse as ph
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'project_variant_2',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [7]:
query = """
SELECT  
        pur.purchase_date AS purchase_date,
        pur.purchase_id AS purchase_id,
        pur.client_id AS client_id,
        dateDiff('year', client.birth_date, toDate(now( ))) AS client_age,
        dateDiff('year', client.registration, toDate(now( ))) AS client_registration_age,
        t.client_category AS client_category,
        pro.promotion_name AS promotion_name, 
        pro.category_name AS category_name,
        pro.partner_name AS partner_name,
        city_client.client_city AS client_city,
        pur_city.city AS city,
        cast(pur.price * pur.quantity AS Float32) AS revenue ,
        cast(pur.quantity AS Int8) AS quantity
        
FROM project_variant_2.promotion AS pro

    RIGHT JOIN project_variant_2.purchase AS pur ON pur.promotion_id = pro.promotion_id
    
    LEFT JOIN project_variant_2.client AS client ON client.client_id = pur.client_id
     
    LEFT JOIN (SELECT DISTINCT client_city_id, client_city
     
                FROM project_variant_2.city) AS city_client ON client.client_city_id = city_client.client_city_id
                
    LEFT JOIN (SELECT DISTINCT city_id, city 
    
                FROM project_variant_2.city) AS pur_city ON pur.city_id = pur_city.city_id
                
    LEFT JOIN (SELECT client_id, CASE
                                      WHEN purchases > 1 THEN 'old'
                                      ELSE 'new'
                                      END AS client_category
                FROM (
                        SELECT client_id, uniqExact(purchase_id) AS purchases
                        FROM project_variant_2.purchase
                        GROUP BY client_id
                     ) AS p)  AS t  ON t.client_id = pur.client_id
WHERE pur.status = '1' AND pur.purchase_date BETWEEN '2020-05-01' AND '2020-08-01'
ORDER BY purchase_date
"""

df = ph.read_clickhouse(query, connection=connection_default)
df

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-05-01,7629,258,27,2,old,campaign_1,Бакалея,Google,Минск,Вена,675.0,5
1,2020-05-01,7585,770,21,2,old,campaign_1,Бакалея,Google,Киев,Ярославль,1395.0,9
2,2020-05-01,6594,210,22,2,old,campaign_1,Бакалея,Google,Минск,Новгород,1330.0,7
3,2020-05-01,6462,601,26,2,old,campaign_1,Бакалея,Google,Казань,Минск,705.0,3
4,2020-05-01,6386,151,25,2,old,campaign_1,Бакалея,Google,Ярославль,Щучинщина,1460.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,2020-08-01,3448,560,23,2,old,campaign_4,Прочее,KarpovCourses,Казань,Ярославль,1575.0,5
2731,2020-08-01,2978,312,27,2,old,campaign_4,Прочее,KarpovCourses,Киев,Ярославль,2200.0,5
2732,2020-08-01,2121,188,22,1,old,campaign_4,Прочее,KarpovCourses,Москва,Москва,495.0,1
2733,2020-08-01,771,336,28,2,old,campaign_4,Прочее,KarpovCourses,Минск,Казань,335.0,1
